In [45]:
import requests
import polars as pl
import pandas as pd

In [ ]:
pl.Config(tbl_rows=10)

test_data = pl.read_csv('../data/User0_credit_card_transactions.csv')
test_data.head()

In [ ]:
def preprocess_dataset(data_df):
    """ Preprocess dataset for service. """
    new_data_df = data_df.with_columns(
        Hour = pl.col("Time").map_elements(
            lambda x: x[:2]).cast(pl.Int64, strict=True),
        Minute = pl.col("Time").map_elements(
            lambda x: x[3:]).cast(pl.Int64, strict=True),
    ).drop("Time").with_columns(
        pl.col("Amount").map_elements(
            lambda x: x.replace("$", "")).cast(pl.Float64, strict=True)
    ).with_columns(
        pl.col("Merchant State").fill_null("ONLINE")
    ).with_columns(
        pl.col("Zip").cast(pl.String, strict=True).fill_null("ONLINE")
    ).with_columns(
        pl.col("Errors?").fill_null(value="No")
    ).with_columns(
        pl.col("Is Fraud?").map_elements(
            lambda x: 0 if x == "No" else 1
        )
    )
    return new_data_df
test_data = preprocess_dataset(test_data)
test_data

In [ ]:
json_data = test_data.to_pandas().to_json(orient='records')


In [65]:
response = requests.post(
    'http://0.0.0.0:3000/predict',
    data=json_data
)

response

<Response [200]>

In [67]:
predictions = response.json()
result = pd.DataFrame.from_dict(predictions, orient='columns')
result

,User,Card,Year,Month,Day,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?,Hour,Minute,is_fraud_proba,is_legit_proba
0,0,0,2002,9,1,134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,5300,No,0,6,21,0.4,0.6
1,0,0,2002,9,1,38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,No,0,6,42,0.3,0.7
2,0,0,2002,9,2,120.34,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,No,0,6,22,0.5,0.5
3,0,0,2002,9,2,128.95,Swipe Transaction,3414527459579106770,Monterey Park,CA,91754.0,5651,No,0,17,45,0.4,0.6
4,0,0,2002,9,3,104.71,Swipe Transaction,5817218446178736267,La Verne,CA,91750.0,5912,No,0,6,23,0.4,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19958,0,4,2009,2,12,9.71,Swipe Transaction,4060646732831064559,La Verne,CA,91750.0,5411,Technical Glitch,0,13,3,0.0,1.0
19959,0,4,2009,2,22,27.94,Swipe Transaction,-8129520121141002186,La Verne,CA,91750.0,5211,No,0,11,58,0.2,0.8
19960,0,4,2009,2,24,13.97,Swipe Transaction,5474320255037684877,Hawthorne,CA,90250.0,5912,No,0,16,53,0.3,0.7
19961,0,4,2009,3,29,14.27,Swipe Transaction,4060646732831064559,La Verne,CA,91750.0,5411,No,0,16,30,0.1,0.9
